# ***Preparation for the required modules***

In [ ]:
!pip install python-terrier #Install Indexind , Retreival , Evaluation Module
!pip install nltk #Install Module That Handle All The NLP Tasks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup

In [ ]:
# Need to install additional terrier package for PRF. It will take around 1 min
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

Cloning into 'terrier-prf'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 196 (delta 52), reused 173 (delta 36), pack-reused 0
Receiving objects: 100% (196/196), 28.00 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaopalliance-java libapache-pom-java libatinject-jsr330-api-java libcdi-api-java
  libcommons-cli-java libcommons-io-java libcommons-lang3-java libcommons-parent-java
  libgeronimo-annotation-1.3-spec-java libgeronimo-interceptor-3.0-spec-java libguava-java
  libguice-java libhawtjni-runtime-java libjansi-java libjansi-native-java libjsr305-java
  libmaven-parent-java libmaven-resolver-java libmaven-shared-utils-java libmaven3-core-java
  libplexus-cipher-java libplexus-classworlds-java libplexus-compone

In [ ]:
import zipfile
import os
import pandas as pd
import re
# import requests
# import xml.etree.ElementTree as ET

import pyterrier as pt
if not pt.started(): pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import *
from nltk.stem.porter import *

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

from flask import Flask, request
import time


pd.set_option('display.max_colwidth', 150)
nltk.download('stopwords')
nltk.download('punkt')

stemmer=PorterStemmer()
stop_words = set(stopwords.words('english'))
elmo = hub.load("https://tfhub.dev/google/elmo/3")

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# ***Data Collection***

In [ ]:
zip_file_name = 'cisi.zip'
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('cisi_dataset')
!ls cisi_dataset

CISI.ALL  CISI.QRY  CISI.REL


In [ ]:
def load_cisi_dataset(data_dir):
    documents_path = os.path.join(data_dir, 'CISI.ALL')
    queries_path = os.path.join(data_dir, 'CISI.QRY')
    qrels_path = os.path.join(data_dir, 'CISI.REL')

    documents_df = read_documents(documents_path)
    queries_df = read_queries(queries_path)
    qrels_df = read_qrels(qrels_path)
    return documents_df, queries_df, qrels_df

# Read documents from CISI.ALL file
def read_documents(documents_path):
    with open(documents_path, 'r') as file:
        lines = file.readlines()
    documents = []
    current_document = None
    for line in lines:
        if line.startswith('.I'):
            if current_document is not None:
                current_document['Text'] = current_document['Text'].split('\t')[0].strip()  # Remove anything after the first tab
                documents.append(current_document)
            current_document = {'ID': line.strip().split()[1], 'Text': ''}
        elif line.startswith('.T'):
            continue
        elif line.startswith('.A') or line.startswith('.B') or line.startswith('.W') or line.startswith('.X'):
            continue
        else:
            current_document['Text'] += line.strip() + ' '

    # Append the last document
    if current_document is not None:
        current_document['Text'] = current_document['Text'].split('\t')[0].strip()  # Remove anything after the first tab
        documents.append(current_document)
    documents_df = pd.DataFrame(documents)
    return documents_df

# Read queries from CISI.QRY file
def read_queries(queries_path):
    with open(queries_path, 'r') as file:
        lines = file.readlines()
    query_texts = []
    query_ids = []
    current_query_id = None
    current_query_text = []
    for line in lines:
        if line.startswith('.I'):
            if current_query_id is not None:
                query_texts.append(' '.join(current_query_text))
                current_query_text = []
            current_query_id = line.strip().split()[1]
            query_ids.append(current_query_id)
        elif line.startswith('.W'):
            continue
        elif line.startswith('.X'):
            break
        else:
            current_query_text.append(line.strip())
    # Append the last query
    query_texts.append(' '.join(current_query_text))
    queries_df = pd.DataFrame({
        'qid': query_ids,
        'query': query_texts})
    return queries_df

# Read qrels from CISI.REL file
def read_qrels(qrels_path):
    qrels_df = pd.read_csv(qrels_path, sep = "\t", header=None)
    qrels_df = qrels_df.iloc[:, [0]]
    qrels_df["qid"] = qrels_df[0].apply(lambda x : x.split()[0])
    qrels_df["docno"] = qrels_df[0].apply(lambda x : x.split()[1])
    qrels_df = qrels_df.drop(columns=[0])
    return qrels_df

In [ ]:
data_dir = '/content/cisi_dataset'
documents_df, queries_df, qrels_df = load_cisi_dataset(data_dir)

In [ ]:
documents_df.head(11)

,ID,Text
0,1,"18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification. The first edi..."
1,2,"Use Made of Technical Libraries Slater, M. This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Librar..."
2,3,"Two Kinds of Power An Essay on Bibliographic Control Wilson, P. The relationships between the organization and control of writings and the organiz..."
3,4,"Systems Analysis of a University Library; final report and research project Buckland, M.K. The establishment of nine new universities in the 1960'..."
4,5,"A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread onl..."
5,6,"Abstracting Concepts and Methods Borko, H. Graduate library school study of abstracting should be more than a how-to-do-it course. It should inclu..."
6,7,"Academic Library Buildings A Guide to Architectural Issues and Solutions Ellsworth, R.E. This book attempts to present representative examples of ..."
7,8,"The Academic Library Essays in Honor of Guy R. Lyle Farber, E.I. As important for staff members' individual development as was the apprenticeship ..."
8,9,"Access to Libraries in College Hyman, R.T. This study assumed that an additional use study held less promise than an analytical consideration of c..."
9,10,"Access to Periodical Resources Palmour, V.E. The purpose of this study was to develop, evaluate, and recommend a national plan for improving acce..."


In [ ]:
queries_df.head(11)

,qid,query
0,1,What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from app...
1,2,"How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information req..."
2,3,What is information science? Give definitions where possible.
3,4,Image recognition and any other methods of automatically transforming printed text into computer-ready form.
4,5,What special training will ordinary researchers and businessmen need for proper information management and unobstructed use of information retriev...
5,6,"What possibilities are there for verbal communication between computers and humans, that is, communication via the spoken word?"
6,7,"Describe presently working and planned systems for publishing and printing original papers by computer, and then saving the byproduct, articles co..."
7,8,Describe information retrieval and indexing in other languages. What bearing does it have on the science in general?
8,9,What possibilities are there for automatic grammatical and contextual analysis of articles for inclusion in an information retrieval system?
9,10,"The use of abstract mathematics in information retrieval, e.g. group theory."


In [ ]:
qrels_df.head(11)

,qid,docno
0,1,28
1,1,35
2,1,38
3,1,42
4,1,43
5,1,52
6,1,65
7,1,76
8,1,86
9,1,150


# ***Preprocessing***

In [ ]:
def Stem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

def clean(text):
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=\@]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = text.strip()
   return text

def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

def preprocess(sentence):
  sentence = clean(sentence)
  sentence = remove_stopwords(sentence)
  sentence = Stem_text(sentence)
  return sentence

**Qrels Preprocessing**

In [ ]:
qrels = pd.DataFrame()
topics = pd.DataFrame()

qrels["qid"] = qrels_df["qid"].iloc[:51]
qrels["docno"] = qrels_df["docno"].iloc[:51]
qrels["label"] = qrels["qid"].apply(lambda x : 1).iloc[:51]

topics["qid"] = queries_df["qid"].iloc[:51]
topics["query"] = queries_df['query'].apply(preprocess).astype(str).apply(lambda x : x.replace("?", "")).iloc[:51]

**Documents Preprocessing**

In [ ]:
documents_df["Processed_text"]=documents_df['Text'].apply(preprocess).apply(clean)
documents_df["docno"] = documents_df["ID"].astype(str)
documents_df.drop(columns='ID',inplace=True)

In [ ]:
documents_df.head(11)

,Text,Processed_text,docno
0,"18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification. The first edi...",18 edit dewey decim classif comaromi j p present studi histori dewey decim classif first edit ddc publish 1876 eighteenth edit 1971 futur edit con...,1
1,"Use Made of Technical Libraries Slater, M. This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Librar...",use made technic librari slater report analysi 6300 act use 104 technic librari unit kingdom librari use one aspect wider pattern inform use infor...,2
2,"Two Kinds of Power An Essay on Bibliographic Control Wilson, P. The relationships between the organization and control of writings and the organiz...",two kind power essay bibliograph control wilson p relationship organ control write organ control knowledg inform inevit enter stori write contain ...,3
3,"Systems Analysis of a University Library; final report and research project Buckland, M.K. The establishment of nine new universities in the 1960'...",system analysi univers librari ; final report research project buckland k establish nine new univers 1960 's provok highli stimul re-examin natur ...,4
4,"A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread onl...",librari manag game report research project brophi p although use game profession educ becom widespread last decad method use number field mani hun...,5
5,"Abstracting Concepts and Methods Borko, H. Graduate library school study of abstracting should be more than a how-to-do-it course. It should inclu...",abstract concept method borko h graduat librari school studi abstract how-to-do-it cours includ gener materi characteristc type abstract histor de...,6
6,"Academic Library Buildings A Guide to Architectural Issues and Solutions Ellsworth, R.E. This book attempts to present representative examples of ...",academ librari build guid architectur issu solut ellsworth r e book attempt present repres exampl success architectur solut import problem librari...,7
7,"The Academic Library Essays in Honor of Guy R. Lyle Farber, E.I. As important for staff members' individual development as was the apprenticeship ...",academ librari essay honor guy r lyle farber e import staff member ' individu develop apprenticeship administr perhap signific attitud one acquir ...,8
8,"Access to Libraries in College Hyman, R.T. This study assumed that an additional use study held less promise than an analytical consideration of c...",access librari colleg hyman r studi assum addit use studi held less promis analyt consider concept basic approach survey compar tradit current pro...,9
9,"Access to Periodical Resources Palmour, V.E. The purpose of this study was to develop, evaluate, and recommend a national plan for improving acce...",access period resourc palmour v e purpos studi develop evalu recommend nation plan improv access period resourc 48 percent academ interlibrari loa...,10


# ***Indexing***

In [ ]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
index_ref = indexer.index(documents_df["Processed_text"], documents_df["docno"])
index = pt.IndexFactory.of(index_ref.toString())

**Built in Method**

In [ ]:
for kv in index.getLexicon(): print( kv.getKey()," -> " ,kv.getValue() )

Streaming output truncated to the last 5000 lines.
district  ->  term2641 Nt=2 TF=2 maxTF=1 @{0 28098 7}
disturb  ->  term5266 Nt=1 TF=1 maxTF=1 @{0 28102 7}
dittmar  ->  term5101 Nt=1 TF=1 maxTF=1 @{0 28105 3}
diver  ->  term1539 Nt=17 TF=20 maxTF=2 @{0 28107 7}
divert  ->  term1865 Nt=1 TF=1 maxTF=1 @{0 28135 2}
divi  ->  term2164 Nt=18 TF=24 maxTF=3 @{0 28137 0}
divid  ->  term1185 Nt=21 TF=29 maxTF=5 @{0 28165 0}
divis  ->  term3958 Nt=1 TF=1 maxTF=1 @{0 28195 4}
divorc  ->  term3340 Nt=1 TF=1 maxTF=1 @{0 28197 6}
dk  ->  term6212 Nt=2 TF=2 maxTF=1 @{0 28200 0}
dna  ->  term7157 Nt=1 TF=2 maxTF=2 @{0 28203 0}
dobrov  ->  term6370 Nt=2 TF=2 maxTF=1 @{0 28205 7}
doc  ->  term5283 Nt=1 TF=1 maxTF=1 @{0 28210 3}
doctor  ->  term3610 Nt=3 TF=4 maxTF=2 @{0 28212 7}
doctrin  ->  term1023 Nt=2 TF=3 maxTF=2 @{0 28220 0}
docu  ->  term1578 Nt=3 TF=4 maxTF=2 @{0 28223 3}
document  ->  term60 Nt=250 TF=563 maxTF=10 @{0 28230 2}
documentalist  ->  term1994 Nt=8 TF=8 maxTF=1 @{0 28427 3}
documen

**DS Map From Scratch**

In [ ]:
TF = {}
for row in documents_df[["docno", "Processed_text"]].values:
    for term in row[1].split():
        if not term in TF: TF[term] = dict()
        if not row[0] in TF[term]: TF[term][row[0]] = 0
        TF[term][row[0]] += 1

for key in sorted(TF.keys()):
    print(f"{key} -> {TF[key]}")

Streaming output truncated to the last 5000 lines.
free-text -> {'492': 1, '523': 2, '813': 1}
freedman -> {'619': 1}
freedom -> {'78': 1, '1438': 1}
freedom-to-read -> {'237': 1}
freeli -> {'927': 1, '942': 1}
freeman -> {'711': 1}
freer -> {'354': 1}
freez -> {'1020': 1}
french -> {'357': 1, '707': 2, '864': 2, '1024': 1, '1108': 1, '1229': 1, '1382': 1, '1414': 1}
frequenc -> {'41': 3, '51': 1, '71': 1, '76': 1, '79': 3, '183': 1, '198': 1, '233': 1, '315': 1, '321': 3, '329': 2, '379': 1, '400': 1, '420': 1, '450': 1, '466': 1, '512': 1, '524': 1, '552': 3, '571': 1, '596': 1, '598': 1, '632': 2, '638': 3, '646': 1, '649': 1, '658': 1, '666': 1, '700': 1, '757': 1, '812': 1, '893': 1, '894': 3, '1072': 1, '1090': 1, '1273': 1, '1301': 1, '1355': 1, '1381': 1, '1396': 1, '1416': 3}
frequency-of-occurr -> {'773': 1}
frequent -> {'14': 1, '16': 1, '23': 1, '32': 1, '59': 1, '183': 1, '198': 2, '210': 1, '228': 1, '233': 1, '294': 1, '312': 1, '318': 1, '357': 1, '364': 1, '422': 1, '4

# ***Query Processing***

In [ ]:
def get_query_ids(query):
    terrier_index = pt.IndexFactory.of(index_ref)
    lex = terrier_index.getLexicon()
    meta = terrier_index.getMetaIndex()
    inv = terrier_index.getInvertedIndex()
    le = lex.getLexiconEntry(query)

    ids = [meta.getItem("docno", posting.getId()) for posting in inv.getPostings(le)]
    doc_descriptions = [documents_df.loc[int(doc_id) - 1, "Text"] for doc_id in ids]
    result_df = pd.DataFrame({"Needed_Document_IDS": ids, "Documents": doc_descriptions})
    return result_df

query=input("Enter Your Query Here : ")
query=preprocess(query)
print(f"Your Preprocessed Query : {query}")

print("                                            ")
print("                                            ")
print("                                            ")
print("                                            ")
print("                                            ")

Enter Your Query Here : parsing
Your Preprocessed Query : pars
                                            
                                            
                                            
                                            
                                            


In [ ]:
get_query_ids(preprocess(query)).head()

,Needed_Document_IDS,Documents
0,1427,"Understanding Media: the extensions of man Winograd, T. When it was decided that Cognitive Psychology would not specify size limits for articles,..."


**Rank the retrieved documents based ranking algorithm (TF-IDF).**

In [ ]:
tfidf_retr = pt.BatchRetrieve(index ,  wmodel = "TF_IDF", num_results = 5)
tfidf_results = tfidf_retr.search(query)

doc_descriptions = [documents_df.loc[int(doc_id) - 1, "Text"] for doc_id in tfidf_results['docno']]

Before_exp_result = pd.DataFrame({
    "Needed_Document_IDS": tfidf_results['docno'],
    "Score": tfidf_results['score'],
    "Documents": doc_descriptions})
Before_exp_result

,Needed_Document_IDS,Score,Documents
0,1427,5.12462,"Understanding Media: the extensions of man Winograd, T. When it was decided that Cognitive Psychology would not specify size limits for articles,..."


# ***Query expansion***

**RM3 query expansion**

In [ ]:
rm3_expander = pt.rewrite.RM3(index)
qe_tfidf = tfidf_retr >> rm3_expander
expanded_q = qe_tfidf.search(query).iloc[0, 2]
expanded_q

'applypipeline:off languag^0.076190472 theori^0.019047618 natur^0.057142857 specifi^0.038095236 semant^0.038095236 uniqu^0.019047618 suffici^0.038095236 intellig^0.038095236 understand^0.038095236 par^0.600000024 artifici^0.038095236'

In [ ]:
expanded_query_formatted = ' '.join(expanded_q.split()[1:])
results_wqe = tfidf_retr.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
pd.concat([tfidf_results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna('')

   Before Expansion    After Expansion


,docid_1,score_1,docid_2,score_2
0,1426.0,5.12462,1426,7.698368
1,,,1224,1.017823
2,,,754,1.015416
3,,,1184,1.002596
4,,,316,0.981138


In [ ]:
tfidf_results = tfidf_retr.search(expanded_q)
doc_descriptions = [documents_df.loc[int(doc_id) - 1, "Text"] for doc_id in tfidf_results['docno']]

After_exp_result= pd.DataFrame({
    "Needed_Document_IDS": tfidf_results['docno'],
    "Score": tfidf_results['score'],
    "Documents": doc_descriptions})
After_exp_result

,Needed_Document_IDS,Score,Documents
0,1427,7.698368,"Understanding Media: the extensions of man Winograd, T. When it was decided that Cognitive Psychology would not specify size limits for articles,..."
1,1225,1.017823,"On semantic synthesis Zholkovskii, A.K. This paper describes a system for the automatic synthesis of a text in a natural language (Russian). The p..."
2,755,1.015416,"The Journal of Documentation Bhattacharyya, K. This paper examines the implications of the findings of evaluative tests regarding the retrieval pe..."
3,1185,1.002596,"Experimentation in the Theory of Linguistic Description Melchuk, I.A. The principle tool for the study and description of natural languages used i..."
4,317,0.981138,"Natural Language Question - Answering System: 1969 Simmons, Robert F. Recent experiments in programming natural language question-answering system..."


**Elmo Synonms**

In [ ]:
def cosine_similarity(emb1, emb2):
    centroid_1 = np.sum(emb1, axis=0) / len(emb1)
    centroid_2 = np.sum(emb2, axis=0) / len(emb2)
    return np.dot(centroid_1, centroid_2) / np.linalg.norm(centroid_1) / np.linalg.norm(centroid_2)

In [ ]:
sentences = [query]
relevent_documents = documents_df[documents_df["docno"].isin(tfidf_results["docno"])]

for i in range(len(relevent_documents )):
    sentences.append(relevent_documents .iloc[i][0])

embeddings = elmo.signatures["default"](tf.constant(sentences))["elmo"]

score = []
for i in range(1, len(sentences)):
    score.append((cosine_similarity(embeddings[0].numpy(), embeddings[i].numpy()), sentences[i]))

score.sort(key=lambda x: x[0], reverse=True)
result = pd.DataFrame()
result['score']  , result['docs'] =  [i[0] for i in score] , [ i[1] for i in score]
result

,score,docs
0,0.607086,"On semantic synthesis Zholkovskii, A.K. This paper describes a system for the automatic synthesis of a text in a natural language (Russian). The p..."
1,0.129242,"Experimentation in the Theory of Linguistic Description Melchuk, I.A. The principle tool for the study and description of natural languages used i..."
2,0.058884,"Understanding Media: the extensions of man Winograd, T. When it was decided that Cognitive Psychology would not specify size limits for articles,..."
3,0.056470,"The Journal of Documentation Bhattacharyya, K. This paper examines the implications of the findings of evaluative tests regarding the retrieval pe..."
4,0.054671,"Natural Language Question - Answering System: 1969 Simmons, Robert F. Recent experiments in programming natural language question-answering system..."


# ***User Interface***

In [ ]:
from google.colab.output import eval_js
print(eval_js('google.colab.kernel.proxyPort(5000)'))

https://xdeoye9aaon-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
app = Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        start_time = time.time()
        query = request.form.get('query')
        tfidf_retr = pt.BatchRetrieve(index ,  wmodel = "TF_IDF", controls = {"c": 0.15}, num_results = 100)
        if query:
            # expanded_q = qe_tfidf.search(preprocess(query)).iloc[0, 2]
            # tfidf_results = tfidf_retr.search(expanded_q)
            tfidf_results = tfidf_retr.search(preprocess(query))
            doc_descriptions = [documents_df.loc[int(doc_id) - 1, "Text"] for doc_id in tfidf_results['docno']]
            end_time = time.time()
            return '''
                <!DOCTYPE html>
                <html>

                <head> <title> Google Meen Ya3m &#129322</title> </head>

                <body>
                    <div class="content">
                        <div class="container">
                            <div class="row">
                                <div class="col-lg-12">
                                    <div class="search-result-box card-box">
                                        <h4>Search Results For "{}" total results {}</h4>
                                        <p>Search time: {:.6f} seconds</p>
                                        <ul>
                                            {}
                                        </ul>
                                    </div>
                                </div>
                            </div>
                        </div>
                    </div>
                </body>
                </html>
            '''.format(query, len(doc_descriptions), end_time - start_time, ''.join(f'<li>{desc}</li>' for desc in doc_descriptions))
        else:
            return "Please provide a query."
    else:
        return '''
            <!DOCTYPE html>
            <html>
            <head> <title>Google Meen Ya3m &#129322</title> </head>

            <body>
                <div class="content">
                    <div class="container">
                        <div class="row">
                            <div class="col-lg-12">
                                <!-- Your search box -->
                                <div class="search-box card-box">
                                    <form method="post">
                                        <input type="text" name="query" placeholder="Search query">
                                        <button type="submit">Search</button>
                                    </form>
                                </div>
                            </div>
                        </div>
                    </div>
                </div>
            </body>
            </html>
        '''

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/May/2024 05:09:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2024 05:09:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/May/2024 05:09:48] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2024 05:09:49] "GET /favicon.ico HTTP/1.1" 404 -


# ***Evaluation***

In [ ]:
topics

,qid,query
0,1,problem concern make descript titl difficulti involv automat retriev articl approxim titl usual relev content articl titl
1,2,actual pertin data oppos refer entir articl retriev automat respons inform request
2,3,inform scienc give definit possibl
3,4,imag recognit method automat transform print text computer-readi form
4,5,special train ordinari research businessmen need proper inform manag unobstruct use inform retriev system problem like encount
5,6,possibl verbal commun comput human commun via spoken word
6,7,describ present work plan system publish print origin paper comput save byproduct articl code data-process form use retriev
7,8,describ inform retriev index languag bear scienc gener
8,9,possibl automat grammat contextu analysi articl inclus inform retriev system
9,10,use abstract mathemat inform retriev e g group theori


In [ ]:
qrels

,qid,docno,label
0,1,28,1
1,1,35,1
2,1,38,1
3,1,42,1
4,1,43,1
5,1,52,1
6,1,65,1
7,1,76,1
8,1,86,1
9,1,150,1


In [ ]:
queries_df['query'][5]

In [ ]:
topics.shape

In [ ]:
qrels['docno'].unique()

In [ ]:
topics["query"] = topics["query"].astype(str)
evaluation = pt.Evaluate(tfidf_retr.transform(topics), qrels, metrics=["map", "recall", "P"], perquery=True)
evaluation_df = pd.DataFrame(evaluation).T
evaluation_df

,map,P@5,P@10,P@15,P@20,P@30,P@100,P@200,P@500,P@1000,R@5,R@10,R@15,R@20,R@30,R@100,R@200,R@500,R@1000
1,0.056522,0.6,0.3,0.2,0.15,0.1,0.03,0.015,0.006,0.003,0.065217,0.065217,0.065217,0.065217,0.065217,0.065217,0.065217,0.065217,0.065217
2,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
